## Bowling Analysis

In [1]:

import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

pio.renderers.default = 'notebook_connected'

# Load datasets
matches = pd.read_csv("../ipl_wrang/ingestion/cleaned_data/matches_cleaned.csv")
deliveries = pd.read_csv("../ipl_wrang/ingestion/cleaned_data/deliveries_cleaned.csv")
ipl_runs = pd.read_csv("../ipl_wrang/data/IPL_last_year_runs.csv")
ipl_wickets = pd.read_csv("../ipl_wrang/data/IPL_last_year_wickets.csv")


### Purple Cap

In [2]:

top_wickets = ipl_wickets[['Player', 'Wickets']].sort_values(by='Wickets', ascending=False).head(10)

fig = px.bar(top_wickets, x='Wickets', y='Player', orientation='h',
             title="Top 10 Wicket Takers", text_auto=True)
fig.update_layout(yaxis={'categoryorder':'total ascending'}, xaxis_title="Wickets", yaxis_title="Bowler")
fig.show()


### Wicket Types

In [3]:

dismissal_counts = deliveries['dismissal_kind'].dropna().value_counts().reset_index()
dismissal_counts.columns = ['dismissal', 'count']

fig = px.pie(dismissal_counts, names='dismissal', values='count', title='Dismissal Types Distribution')
fig.show()


### Spider Chart

In [4]:
import pandas as pd
import plotly.graph_objects as go
import plotly.io as pio

pio.renderers.default = 'notebook_connected'


bowlers = ['JJ Bumrah', 'R Ashwin', 'YS Chahal'] 

filtered = deliveries[deliveries['bowler'].isin(bowlers)]

bowler_stats = filtered.groupby('bowler').agg({
    'match_id': 'nunique',
    'total_runs': 'sum'
}).reset_index()

bowler_stats['balls_bowled'] = filtered.groupby('bowler').size().values

wickets = filtered[(filtered['is_wicket'] == 1) & (~filtered['dismissal_kind'].isin(['run out']))].groupby('bowler').size()
bowler_stats['wickets'] = bowler_stats['bowler'].map(wickets).fillna(0).astype(int)

bowler_stats['overs'] = bowler_stats['balls_bowled'] // 6 + (bowler_stats['balls_bowled'] % 6) / 6
bowler_stats['economy'] = bowler_stats['total_runs'] / bowler_stats['overs']

bowler_stats['average'] = bowler_stats.apply(
    lambda row: row['total_runs'] / row['wickets'] if row['wickets'] > 0 else 0,
    axis=1
)

bowler_stats['strike_rate'] = bowler_stats.apply(
    lambda row: row['balls_bowled'] / row['wickets'] if row['wickets'] > 0 else 0,
    axis=1
)

bowler_stats['norm_wickets'] = bowler_stats['wickets'] / bowler_stats['wickets'].max() * 100
bowler_stats['norm_economy'] = bowler_stats['economy'].min() / bowler_stats['economy'] * 100  
bowler_stats['norm_average'] = bowler_stats['average'].min() / bowler_stats['average'] * 100  
bowler_stats['norm_balls'] = bowler_stats['balls_bowled'] / bowler_stats['balls_bowled'].max() * 100
bowler_stats['norm_strike_rate'] = bowler_stats['strike_rate'].min() / bowler_stats['strike_rate'] * 100  

fig = go.Figure()

for i in range(len(bowler_stats)):
    fig.add_trace(go.Scatterpolar(
        r=[
            bowler_stats.loc[i, 'norm_wickets'],
            bowler_stats.loc[i, 'norm_economy'],
            bowler_stats.loc[i, 'norm_average'],
            bowler_stats.loc[i, 'norm_balls'],
            bowler_stats.loc[i, 'norm_strike_rate']
        ],
        theta=['Wickets', 'Economy', 'Average', 'Balls Bowled', 'Strike Rate'],
        fill='toself',
        name=bowler_stats.loc[i, 'bowler']
    ))

fig.update_layout(
    polar=dict(
        radialaxis=dict(visible=True, range=[0, 100])
    ),
    title="Bowler Performance Radar Chart (with Strike Rate)",
    showlegend=True
)

fig.show()